In [4]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt 
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

TICKER_DIR = "C:\\Users\\anton\\Documents\\antoniouaa\\msc_thesis\\data\\tickers\\ticker_data"

ticker = "WFC"

dataset = pd.read_csv(os.path.join(TICKER_DIR, f"{ticker}.csv"))
dataset = dataset.dropna(how="any")
close = dataset["close"].tail(1000).values

sc = MinMaxScaler()
scaled = sc.fit_transform(np.reshape(close, (-1, 1)))

In [5]:
def evaluate_arima(X, order):
    size = int(len(X) * 0.8)
    train, test = X[:size], X[size:]
    history = [x for x in train]
    predictions = []
    # perform walk-forward validation
    for t, _ in enumerate(test):
        # fit arima model to the history of values
        model = ARIMA(history, order=order)
        model_fit = model.fit(disp=-1)
        # predict the next value
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])

    mse = mean_squared_error(test, predictions)
    mae = mean_absolute_error(test, predictions)
    return mse, mae, model

In [6]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning, HessianInversionWarning
warnings.simplefilter("ignore", ConvergenceWarning)
warnings.simplefilter("ignore", HessianInversionWarning)

order = (2, 2, 0)

mse, mae, model = evaluate_arima(scaled, order)
print(f"MSE: {mse}\nMAE: {mae}")

MSE: 0.0010332794267077677
MAE: 0.025804764092524014


In [7]:
model.summary()

AttributeError: 'ARIMA' object has no attribute 'summary'

In [148]:
# with open("scores_arima.csv", "a") as scores_file:
#     scores_file.write(f"{ticker},{mae},{mse}\n")

In [149]:
def evaluate_models(X, p_values, d_values, q_values):
    scores = []
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p, d, q)
                try:
                    mse = evaluate_arima(X, order)
                    scores.append((mse, order))
                    print(f"ARIMA ORDER: {order}\tMSE: {mse}")
                except:
                    continue
    return scores

In [150]:
scores = evaluate_models(X, [0, 1, 2], range(2), range(2))
best_score = sorted(scores)[0]
print(f"BEST SCORE: {best_score[0]}\tORDER: {best_score[1]}")